In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1515835,2021-07-31,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1515836,2021-08-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1515837,2021-08-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1515838,2021-08-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
39859,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39861,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39863,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39865,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
39867,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
926717,2021-07-31,Arkansas,Arkansas,5001,2571,41.00,5000,Arkansas,AR,Arkansas,State,3017804
926719,2021-08-01,Arkansas,Arkansas,5001,2591,41.00,5000,Arkansas,AR,Arkansas,State,3017804
926721,2021-08-02,Arkansas,Arkansas,5001,2594,42.00,5000,Arkansas,AR,Arkansas,State,3017804
926723,2021-08-03,Arkansas,Arkansas,5001,2604,42.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1515835,2021-07-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1515836,2021-08-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1515837,2021-08-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1515838,2021-08-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-08-04') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
561,2021-08-04,Snohomish,Washington,43771,623.00,53061,WA,County,822083,5324.40,75.78
1120,2021-08-04,Cook,Illinois,566985,11136.00,17031,IL,County,5150233,11008.92,216.22
1678,2021-08-04,Orange,California,285620,5146.00,6059,CA,County,3175692,8993.95,162.04
2235,2021-08-04,Maricopa,Arizona,588243,10485.00,4013,AZ,County,4485414,13114.58,233.76
2792,2021-08-04,Los Angeles,California,1312352,24721.00,6037,CA,County,10039107,13072.40,246.25
...,...,...,...,...,...,...,...,...,...,...,...
1514779,2021-08-04,Wheeler,Oregon,49,1.00,41069,OR,County,1332,3678.68,75.08
1515075,2021-08-04,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1515340,2021-08-04,Esmeralda,Nevada,41,0.00,32009,NV,County,873,4696.45,0.00
1515601,2021-08-04,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
561,2021-08-04,Snohomish,Washington,43771,623.00,53061,WA,County,822083,5324.40,75.78,75.78,5324.40
1120,2021-08-04,Cook,Illinois,566985,11136.00,17031,IL,County,5150233,11008.92,216.22,216.22,11008.92
1678,2021-08-04,Orange,California,285620,5146.00,6059,CA,County,3175692,8993.95,162.04,162.04,8993.95
2235,2021-08-04,Maricopa,Arizona,588243,10485.00,4013,AZ,County,4485414,13114.58,233.76,233.76,13114.58
2792,2021-08-04,Los Angeles,California,1312352,24721.00,6037,CA,County,10039107,13072.40,246.25,246.25,13072.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514779,2021-08-04,Wheeler,Oregon,49,1.00,41069,OR,County,1332,3678.68,75.08,75.08,3678.68
1515075,2021-08-04,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1515340,2021-08-04,Esmeralda,Nevada,41,0.00,32009,NV,County,873,4696.45,0.00,0.00,4696.45
1515601,2021-08-04,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00,0.00,1775.15


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
969206,2021-08-04,Hale,Alabama,2369,79.00,1065,AL,County,14651,16169.54,539.21,539.21,16169.54,1
1055401,2021-08-04,Clarke,Alabama,3746,63.00,1025,AL,County,23622,15858.10,266.70,266.70,15858.10,2
757581,2021-08-04,Lowndes,Alabama,1465,55.00,1085,AL,County,9726,15062.72,565.49,565.49,15062.72,3
563076,2021-08-04,Franklin,Alabama,4656,81.00,1059,AL,County,31362,14845.99,258.27,258.27,14845.99,4
671433,2021-08-04,Etowah,Alabama,15106,372.00,1055,AL,County,102268,14770.99,363.75,363.75,14770.99,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815233,2021-08-04,Hot Springs,Wyoming,387,3.00,56017,WY,County,4413,8769.54,67.98,67.98,8769.54,19
924748,2021-08-04,Converse,Wyoming,1176,20.00,56009,WY,County,13822,8508.18,144.70,144.70,8508.18,20
1150953,2021-08-04,Lincoln,Wyoming,1542,14.00,56023,WY,County,19830,7776.10,70.60,70.60,7776.10,21
1281989,2021-08-04,Niobrara,Wyoming,173,3.00,56027,WY,County,2356,7342.95,127.33,127.33,7342.95,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
757581,2021-08-04,Lowndes,Alabama,1465,55.00,1085,AL,County,9726,15062.72,565.49,565.49,15062.72,3,1
969206,2021-08-04,Hale,Alabama,2369,79.00,1065,AL,County,14651,16169.54,539.21,539.21,16169.54,1,2
817218,2021-08-04,Greene,Alabama,999,37.00,1063,AL,County,8111,12316.61,456.17,456.17,12316.61,34,3
321334,2021-08-04,Walker,Alabama,7873,288.00,1127,AL,County,63521,12394.33,453.39,453.39,12394.33,33,4
669441,2021-08-04,Clay,Alabama,1676,59.00,1027,AL,County,13235,12663.39,445.79,445.79,12663.39,25,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150953,2021-08-04,Lincoln,Wyoming,1542,14.00,56023,WY,County,19830,7776.10,70.60,70.60,7776.10,21,19
1122102,2021-08-04,Uinta,Wyoming,2464,14.00,56041,WY,County,20226,12182.34,69.22,69.22,12182.34,4,20
815233,2021-08-04,Hot Springs,Wyoming,387,3.00,56017,WY,County,4413,8769.54,67.98,67.98,8769.54,19,21
319822,2021-08-04,Teton,Wyoming,3938,11.00,56039,WY,County,23464,16783.16,46.88,46.88,16783.16,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,75.78,5324.40,20,24
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,75.78,5324.40,20,24
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,75.78,5324.40,20,24
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,75.78,5324.40,20,24
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,75.78,5324.40,20,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512307,2021-07-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1512308,2021-08-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1512309,2021-08-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1512310,2021-08-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
965186,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16169.54,2,1,1.00
965187,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16169.54,2,1,0.00
965188,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16169.54,2,1,0.00
965189,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16169.54,2,1,0.00
965190,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,539.21,16169.54,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170357,2021-07-31,Crook,Wyoming,476,13.00,56011,WY,County,7584,6276.37,171.41,171.41,6355.49,7,23,0.00
1170358,2021-08-01,Crook,Wyoming,476,13.00,56011,WY,County,7584,6276.37,171.41,171.41,6355.49,7,23,0.00
1170359,2021-08-02,Crook,Wyoming,481,13.00,56011,WY,County,7584,6342.30,171.41,171.41,6355.49,7,23,5.00
1170360,2021-08-03,Crook,Wyoming,482,13.00,56011,WY,County,7584,6355.49,171.41,171.41,6355.49,7,23,1.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
754547,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15062.72,1,3,1.00,0.00
754548,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15062.72,1,3,0.00,0.00
754549,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15062.72,1,3,0.00,0.00
754550,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15062.72,1,3,0.00,0.00
754551,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15062.72,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752057,2021-07-31,Albany,Wyoming,4774,18.00,56001,WY,County,38880,12278.81,46.30,46.30,12358.54,23,3,0.00,0.00
752058,2021-08-01,Albany,Wyoming,4774,18.00,56001,WY,County,38880,12278.81,46.30,46.30,12358.54,23,3,0.00,0.00
752059,2021-08-02,Albany,Wyoming,4788,18.00,56001,WY,County,38880,12314.81,46.30,46.30,12358.54,23,3,14.00,0.00
752060,2021-08-03,Albany,Wyoming,4798,18.00,56001,WY,County,38880,12340.53,46.30,46.30,12358.54,23,3,10.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-08-04') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
383798,2021-08-04,Imperial,California,32277,744.00,6025,CA,County,181215,17811.44,410.56,410.56,17811.44,1,2,31.00,2.00
2792,2021-08-04,Los Angeles,California,1312352,24721.00,6037,CA,County,10039107,13072.40,246.25,246.25,13072.40,2,5,3686.00,16.00
190728,2021-08-04,San Bernardino,California,313044,5265.00,6071,CA,County,2180085,14359.26,241.50,241.50,14359.26,3,4,386.00,12.00
673377,2021-08-04,Inyo,California,1473,38.00,6027,CA,County,18039,8165.64,210.65,210.65,8165.64,4,30,14.00,0.00
85105,2021-08-04,Stanislaus,California,66619,1097.00,6099,CA,County,550660,12098.03,199.22,199.22,12098.03,5,9,143.00,0.00
67665,2021-08-04,San Joaquin,California,81736,1474.00,6077,CA,County,762148,10724.43,193.40,193.40,10724.43,6,15,168.00,13.00
39342,2021-08-04,Riverside,California,318968,4667.00,6065,CA,County,2470546,12910.83,188.91,188.91,12910.83,7,7,743.00,0.00
105582,2021-08-04,Tulare,California,60807,854.00,6107,CA,County,466195,13043.25,183.19,183.19,13043.25,8,6,89.00,0.00
38310,2021-08-04,Fresno,California,115322,1746.00,6019,CA,County,999101,11542.58,174.76,174.76,11542.58,9,12,261.00,2.00
610021,2021-08-04,Merced,California,33321,485.00,6047,CA,County,277680,11999.78,174.66,174.66,11999.78,10,10,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1431979,2021-08-04,Lassen,California,5905,26.00,6035,CA,County,30573,19314.43,85.04,85.04,19314.43,35,1,7.00,5.00
383798,2021-08-04,Imperial,California,32277,744.00,6025,CA,County,181215,17811.44,410.56,410.56,17811.44,1,2,31.00,2.00
817160,2021-08-04,Kings,California,25050,252.00,6031,CA,County,152940,16378.97,164.77,164.77,16378.97,11,3,58.00,1.00
190728,2021-08-04,San Bernardino,California,313044,5265.00,6071,CA,County,2180085,14359.26,241.50,241.50,14359.26,3,4,386.00,12.00
2792,2021-08-04,Los Angeles,California,1312352,24721.00,6037,CA,County,10039107,13072.40,246.25,246.25,13072.40,2,5,3686.00,16.00
105582,2021-08-04,Tulare,California,60807,854.00,6107,CA,County,466195,13043.25,183.19,183.19,13043.25,8,6,89.00,0.00
39342,2021-08-04,Riverside,California,318968,4667.00,6065,CA,County,2470546,12910.83,188.91,188.91,12910.83,7,7,743.00,0.00
239949,2021-08-04,Kern,California,114342,1428.00,6029,CA,County,900202,12701.82,158.63,158.63,12701.82,13,8,263.00,1.00
85105,2021-08-04,Stanislaus,California,66619,1097.00,6099,CA,County,550660,12098.03,199.22,199.22,12098.03,5,9,143.00,0.00
610021,2021-08-04,Merced,California,33321,485.00,6047,CA,County,277680,11999.78,174.66,174.66,11999.78,10,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
383798,2021-08-04,Imperial,California,32277,744.00,6025,CA,County,181215,17811.44,410.56,410.56,17811.44,1,2,31.00,2.00
2792,2021-08-04,Los Angeles,California,1312352,24721.00,6037,CA,County,10039107,13072.40,246.25,246.25,13072.40,2,5,3686.00,16.00
190728,2021-08-04,San Bernardino,California,313044,5265.00,6071,CA,County,2180085,14359.26,241.50,241.50,14359.26,3,4,386.00,12.00
673377,2021-08-04,Inyo,California,1473,38.00,6027,CA,County,18039,8165.64,210.65,210.65,8165.64,4,30,14.00,0.00
85105,2021-08-04,Stanislaus,California,66619,1097.00,6099,CA,County,550660,12098.03,199.22,199.22,12098.03,5,9,143.00,0.00
67665,2021-08-04,San Joaquin,California,81736,1474.00,6077,CA,County,762148,10724.43,193.40,193.40,10724.43,6,15,168.00,13.00
39342,2021-08-04,Riverside,California,318968,4667.00,6065,CA,County,2470546,12910.83,188.91,188.91,12910.83,7,7,743.00,0.00
105582,2021-08-04,Tulare,California,60807,854.00,6107,CA,County,466195,13043.25,183.19,183.19,13043.25,8,6,89.00,0.00
38310,2021-08-04,Fresno,California,115322,1746.00,6019,CA,County,999101,11542.58,174.76,174.76,11542.58,9,12,261.00,2.00
610021,2021-08-04,Merced,California,33321,485.00,6047,CA,County,277680,11999.78,174.66,174.66,11999.78,10,10,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5042,07/31/21,Lassen,5888,26.00,19258.82,85.04,35,1,7.00,0.00
5043,08/01/21,Lassen,5888,26.00,19258.82,85.04,35,1,0.00,0.00
5044,08/02/21,Lassen,5888,26.00,19258.82,85.04,35,1,0.00,0.00
5045,08/03/21,Lassen,5898,21.00,19291.53,68.69,35,1,10.00,-5.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5128,07/31/21,Merced,33321,485.00,11999.78,174.66,10,10,0.00,0.00
5129,08/01/21,Merced,33321,485.00,11999.78,174.66,10,10,0.00,0.00
5130,08/02/21,Merced,33321,485.00,11999.78,174.66,10,10,0.00,0.00
5131,08/03/21,Merced,33321,485.00,11999.78,174.66,10,10,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)